In [28]:
import os
import re
import time
import string
import polars as pl

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
<h3 align="left"><font color='#DEB887'>Info</font></h3>
    
This notebook allows you to concatenate and filter the data necessary to train the Machine Learning model for the Recommendation System

# List all reviews parquet files related to reviews to be loaded

In [5]:
path = '../data/raw/reviews'
with os.scandir(path) as reviewFiles:
    reviewFiles = [file.name for file in reviewFiles if file.is_file()]
print(reviewFiles)

['processed_reviews_books_1.parquet', 'processed_reviews_books_10.parquet', 'processed_reviews_books_11.parquet', 'processed_reviews_books_12.parquet', 'processed_reviews_books_13.parquet', 'processed_reviews_books_3.parquet', 'processed_reviews_books_4.parquet', 'processed_reviews_books_5.parquet', 'processed_reviews_books_6.parquet', 'processed_reviews_books_7.parquet', 'processed_reviews_books_8.parquet', 'processed_reviews_books_9.parquet']


# Load and join all parquet files in the directory

In [6]:
df = pl.DataFrame()
for f in reviewFiles:
    n = str.format("../data/raw/reviews/{}", f)
    print("reading " + n)

    df_aux = pl.read_parquet(n)

    print("Adding..." + n)
    df = df.vstack(df_aux)
    n = ""


reading ../data/raw/reviews/processed_reviews_books_1.parquet
Adding...../data/raw/reviews/processed_reviews_books_1.parquet
reading ../data/raw/reviews/processed_reviews_books_10.parquet
Adding...../data/raw/reviews/processed_reviews_books_10.parquet
reading ../data/raw/reviews/processed_reviews_books_11.parquet
Adding...../data/raw/reviews/processed_reviews_books_11.parquet
reading ../data/raw/reviews/processed_reviews_books_12.parquet
Adding...../data/raw/reviews/processed_reviews_books_12.parquet
reading ../data/raw/reviews/processed_reviews_books_13.parquet
Adding...../data/raw/reviews/processed_reviews_books_13.parquet
reading ../data/raw/reviews/processed_reviews_books_3.parquet
Adding...../data/raw/reviews/processed_reviews_books_3.parquet
reading ../data/raw/reviews/processed_reviews_books_4.parquet
Adding...../data/raw/reviews/processed_reviews_books_4.parquet
reading ../data/raw/reviews/processed_reviews_books_5.parquet
Adding...../data/raw/reviews/processed_reviews_books_5.

In [7]:
df.sample(1)

asin,overall,reviewText,reviewerID,summary,verified,unixReviewTime
str,str,str,str,str,str,datetime[μs]
"""0316055433""","""4.0""","""My response in…","""A2VTZG179DNMJK…","""Excellent piec…","""true""",2015-01-23 00:00:00


In [8]:
df.shape

(2620482, 7)

In [9]:
df.null_count()

asin,overall,reviewText,reviewerID,summary,verified,unixReviewTime
u32,u32,u32,u32,u32,u32,u32
0,0,1260,0,796,0,0


In [10]:
df = df.drop_nulls()

In [11]:
df.shape

(2618468, 7)

Save it as temporal parquet file

In [12]:
df.write_parquet("../data/processed/All_reviews.parquet")

In [13]:
reviews = pl.read_parquet("../data/processed/All_reviews.parquet")

Load the metadata related to books category

In [14]:
metadata = pl.read_parquet("../data/raw/all_metadata_books_in_top25k.parquet")

In [15]:
reviews.shape

(2618468, 7)

In [16]:
metadata.shape

(12049, 9)

In [17]:
reviews.sample(1)

asin,overall,reviewText,reviewerID,summary,verified,unixReviewTime
str,str,str,str,str,str,datetime[μs]
"""0439023521""","""5.0""","""It was amazing…","""AQQHPKCCRC3A8""","""Final book""","""true""",2014-03-19 00:00:00


In [18]:
metadata.sample(1)

asin,brand,price,title,cant_image,cant_category,rank_in_category,also_buy_count,also_view_count
str,str,f64,str,u32,u32,i64,u32,u32
"""0316098809""","""Visit Amazon's…",10.87,"""The Rook""",0,3,23644,99,60


Join 2 dataframes by the ASIN number

In [19]:
join_keys = ["asin"]
df_joined = metadata.join(reviews, join_keys)

In [20]:
df_joined.shape

(2618468, 15)

In [21]:
df_joined.sample(1)

asin,brand,price,title,cant_image,cant_category,rank_in_category,also_buy_count,also_view_count,overall,reviewText,reviewerID,summary,verified,unixReviewTime
str,str,f64,str,u32,u32,i64,u32,u32,str,str,str,str,str,datetime[μs]
"""0439023521""","""Visit Amazon's…",5.0,"""The Hunger Gam…",0,3,2069,67,54,"""5.0""","""I loved""","""A300UNWZW4VWNK…","""Five Stars""","""true""",2016-05-05 00:00:00


In [22]:
final = df_joined.select("title", "reviewerID", "unixReviewTime")


In [23]:
final.sample(1)

title,reviewerID,unixReviewTime
str,str,datetime[μs]
"""In Defense of …","""A1G95M7TH5FXE9…",2014-11-09 00:00:00


In [24]:
final.shape
#2916355

(2618468, 3)

In [25]:
final = final.unique()

In [26]:
final.shape

(2612701, 3)

Save the file to be used

In [27]:
final.write_parquet("../data/processed/Final_data_for_ML.parquet")

In [32]:
# final = final.with_columns(
#     pl.col("unixReviewTime").appy(lambda  x: time.mktime(x.timetuple()) ).alias("revietime")
# )

AttributeError: 'Expr' object has no attribute 'appy'